In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
# %load BusinessRules.py
import Lib
import _StaticFunctions
import _BooleanReturnFunctions
import _AssignFunction
from datetime import datetime
import pandas as pd

# comment below two for local testing
#from ace_logger import Logging
#logging = Logging()

# uncomment these below lines for local testing
import logging 
logger=logging.getLogger() 
logger.setLevel(logging.ERROR) 



@Lib.add_methods_from(_StaticFunctions, _BooleanReturnFunctions, _AssignFunction) 
class BusinessRules():
    
    def __init__(self, case_id, rules, table_data, decision = False):
        self.case_id = case_id
        self.rules = rules
        self.data_source = table_data
        self.is_decision = decision

        # fields which we are maintaining
        self.changed_fields = {}
        self.params_data = {}
        self.params_data['input'] = []

    def evaluate_business_rules(self):
        """Evaluate all the rules"""
        for rule in self.rules:
            logging.info("\n Evaluating the rule: " +f"{rule} \n")
            output = self.evaluate_rule(rule)
            logging.info("\n Output: " +f"{output} \n")
        # update the changes fields in database
        logging.info(f"\nchanged fields are \n{self.changed_fields}\n")
        return self.changed_fields
    
    def evaluate_rule(self,rule):
        """Evaluate the rule"""
        logging.info(f"\nEvaluating the rule \n{rule}\n")

        rule_type = rule['rule_type']
        
        if  rule_type == 'static':
            function_name = rule['function']
            parameters = rule['parameters']
            return  self.evaluate_static(function_name, parameters)
    
        if rule_type == 'condition':
            evaluations = rule['evaluations']
            return self.evaluate_condition(evaluations)
    
    def conditions_met(self, conditions):
        """Evaluate the conditions and give out the final decisoin
        
        """
        eval_string = ''
        # return True if there are no conditions...that means we are doing else..
        if not conditions:
            return True
        # evaluate the conditions
        for condition in conditions:
            logging.info(f"Evaluting the condition {condition}")
            if condition == 'AND' or condition == 'OR':
                eval_string += ' '+condition.lower()+' '
            else:
                eval_string += ' '+str(self.evaluate_rule(condition))+' '
        logging.info(f"\n eval string is {eval_string} \n output is {eval(eval_string)}")
        return eval(eval_string)

    def evaluate_condition(self, evaluations):
        """Execute the conditional statements.

        Args:
            evaluations(dict) 
        Returns:
            decision(boolean) If its is_decision.
            True If conditions met and it is done with executions.
            False For any other case (scenario).
        """
        for each_if_conditions in evaluations:
            conditions = each_if_conditions['conditions']
            executions = each_if_conditions['executions']
            logging.info(f'\nconditions got are \n{conditions}\n')
            logging.info(f'\nexecutions got are \n{executions}\n')
            decision = self.conditions_met(conditions)
            
            """
            Why this self.is_decision and decision ?
                In decison tree there are only one set of conditions to check
                But other condition rules might have (elif conditions which needs to be evaluate) 
            """
            if self.is_decision:
                if decision:
                    for rule in executions:
                        self.evaluate_rule(rule)
                logging.info(f"\n Decision got for the (for decision tree) condition\n {decision}")    
                return decision
            if decision:
                for rule in executions:
                    self.evaluate_rule(rule)
                return True
        return False

    def get_param_value(self, param_object):
        """Returns the parameter value.

        Args:
            param_object(dict) The param dict from which we will parse and get the value.
        Returns:
            The value of the parameter
        Note:
            It returns a value of type we have defined. 
            If the parameter is itself a rule then it evaluates the rule and returns the value.
        """
        logging.info(f"\nPARAM OBJECT IS {param_object}\n")
        param_source = param_object['source']
        if param_source == 'input_config':
            table_key = param_object['table']
            column_key = param_object['column']
            table_key = table_key.strip() # strip for extra spaces
            column_key = column_key.strip() # strip for extra spaces
            logging.debug(f"\ntable is {table_key} and column key is {column_key}\n")
            try:
                data = {}
                # update params data
                data['type'] = 'from_table'
                data['table'] = table_key
                data['column'] = column_key
                data['value'] = self.data_source[table_key][column_key]
                self.params_data['input'].append(data)
                return data['value']
            except Exception as e:
                logging.error(f"\ntable or column key not found\n")
                logging.error(str(e))
                logging.info(f"\ntable data is {self.data_source}\n")
        if param_source == 'rule':
            param_value = param_object['value']
            return self.evaluate_rule(param_value)
        if param_source == 'input':
            param_value = param_object['value']
#             param_value = str(param_value).strip() # converting into strings..need to check
            return  param_value

##############################################################################################################

read_rule = {'rule_type':'static',
              'function':'Read',
              'parameters':{'path':'D:\\AlgonoX\\Python\\BUSINESS RULES\\KARVY_BusinessRules\\sample.csv'}
}

DR_CR_D_check = {'rule_type': 'static',
        'function': 'CompareKeyValue',
        'parameters': {'left_param':{'source': 'rule', 'value':read_rule},
                       'operator':'==',
                       'right_param':{'source':'input', 'value':'D'}
                      }
       }

Amount_Debit = {'rule_type': 'static',
                'function': 'Transform',
                'parameters':[
            {'param':{'source':'rule', 'value':read_rule}},
            {'operator':'*'},
            {'param':{'source':'input', 'value':"-1"}}
        ]
}


Amount_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'Amount'}, 
                                                'assign_value':{'source':'rule', 'value':Amount_Debit}
                      }
}

updating_Amount = {'rule_type': 'condition',
                    'evaluations': [{ 'conditions':[DR_CR_D_check],'executions':[Amount_Assign_rule]

        }]
}


########################################### Assign_Rules ###############################################################################

Feed_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'Feed'}, 
                                                'assign_value':{'source':'input', 'value':'CMS'}
                      }
}

Sub_Feed_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'Sub_Feed'}, 
                                                'assign_value':{'source':'input', 'value':'CMS'}
                      }
}

ID_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'ID'}, 
                                                'assign_value':{'source':'input', 'value':'0'}
                      }
}

Feed_ID_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'karvy_feed_copy', 'column':'Feed_ID'}, 
                                                'assign_value':{'source':'input', 'value':''}
                      }
}

########################################### Filtering_rule ###################################################################

Code_check = {'rule_type': 'static',
        'function': 'CompareKeyValue',
        'parameters': {'left_param':{'source': 'input_config', 'table': 'karvy_feed_copy', 'column': 'Code'},
                       'operator':'==',
                       'right_param':{'source':'input', 'value':''}
                      }
       }

Queue_Assign_rule = {'rule_type': 'static',
                        'function': 'Assign',
                        'parameters': {'assign_table':{'table':'Process_queue', 'column':'queue'}, 
                                                'assign_value':{'source':'input', 'value':'Maker'}
                      }
}    

updating_Queue_Code = {'rule_type': 'condition',
                    'evaluations': [{ 'conditions':[Code_check],'executions':[Queue_Assign_rule]

        }]
}

########################################### Filtering_Date ###################################################################
Filtering_Date = { 'rule_type': 'static',
    'function'  : 'InCheckDate',
    'parameters':{'start_date':{'source':'input', 'value':'2009-01-01'},
                'end_date':{'source':'input', 'value':'2029-01-01'},
                'date':{'source':'input_config','table': 'karvy_feed_copy', 'column': 'Date'}
                }
}

###################################################################################################################################


db_tables = {
                    "karvy_extraction" : ["karvy_feed_copy"],
                    #"queues":["process_queue"]
                }
unique_id = 'Cms_10'

###################################################################################################################################
###################################### Based on Dataframe #########################################################################
"""data = {
    'NAME':['Rachel','Ross','joy','Monica','Phoebe'],
    'ROLL':[497,498,499,500,501],
    'AGE':[23,24,25,26,27]
}

data = pd.DataFrame(data)"""

####################################################################################################################################
count_df_rule = {'rule_type':'static',
                  'function':'Contains',
                  'parameters':{'table_name':'data','column_name':'NAME','value':{'source':'input','value':'Rachel'}}
}

assign_df_rule = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'data', 'column':'ROLL'}, 
                                'assign_value':{'source':'input', 'value':'roll is'}
}
}
#####################################################################################################################################
############################################# Karvy_BusinessRules #########################################################
#####################################################################################################################################

read_rule = {'rule_type':'static',
              'function':'Read',
              'parameters':{'path':'D:\\AlgonoX\\Python\\BUSINESS RULES\\KARVY_BusinessRules\\sample.csv'}
}

#####################################################################################################################################
filtering_rule = {'rule_type': 'static',
                  'function': 'Filter',
                  'parameters': {
                  'from_table': {'source':'input','value':'ocr'},
                 'lookup_filters':[
                {
                    'column_name': 'Amount',
                    'compare_with':  {'source':'input', 'value':1000},
                    'operator':'&'
                },
                {
                    'column_name': 'Plan Code',
                    'compare_with':  {'source':'input', 'value':'AG'},
                    'operator':'|'
                }
            ]
        }

}
########################################## 05-10-2019 #######################################################################

# ocr = pd.read_csv('D:\\AlgonoX\\Python\\BUSINESS RULES\\KARVY_BusinessRules\\sample.csv')

amount_transform = {'rule_type': 'static',
                'function': 'TransformDF',
                'parameters':{
                                'table':'ocr',
                                'value1_colomn': {'source':'input', 'value': 'Amount'},
                                'operator':'*',
                                'value2': {'source':'input', 'value': -1}
                            }
}

# ignore filter_test rule
filter_test = {'rule_type': 'static',
    'function': 'Filter',
    'parameters': {
            'from_table': 'ocr',
            'lookup_filters':[
                {
                    'column_name': 'Amount',
                    'lookup_operator' : '==',
                    'compare_with':  {'source':'input', 'value': 10}
                }
            ]
        }
}

where_test = {'rule_type': 'static',
    'function': 'WhereClause',
    'parameters': {
            'data_frame':{'source':'rule', 'value': Amount_Debit},
            'from_table': 'ocr',
            'colomn':'Amount',
            'lookup_filters':[
                {
                    'column_name': 'Plan Code',
                    'lookup_operator' : '==',
                    'compare_with':  {'source':'input', 'value': 'GP'}
                }
                
            ]

        }
}


###########################################################################################################
read_rule = {'rule_type':'static',
              'function':'Read',
              'parameters':{
                            'path':{'source':'input','value':'C:\\Users\\Algonox\\Desktop\\AlgonoxWork\\BusinessRulesModule\\run_business_rule\\sample.csv'},
                           'table_name':{'source':'input','value':'ocr'}}
}

#####################################################################################################################################
add_filetered_df_rule = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'Filtered'}, 
                                'assign_value':{'source':'input', 'value':'N'}
                }
             }

#################################################################################################################################
add_matched_assign_df_rule = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'Matched'}, 
                                'assign_value':{'source':'input', 'value':'0'}
                }
             }
########################################################################################################################
add_dr_cr_assign_df_rule = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'DR_CR'}, 
                                'assign_value':{'source':'input', 'value':'D'}
                }
             }
#######################################################################################################################
add_amount = {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'Amount'}, 
                                'assign_value':{'source':'input', 'value':50}
                }
             }
#######################################################################################################################

amount_transform = {'rule_type': 'static',
                'function': 'TransformDF',
                'parameters':{
                                'table':'ocr',
                                'value1_colomn': {'source':'input', 'value': 'Amount'},
                                'operator':'*',
                                'value2': {'source':'input', 'value': -1}
                            }
}

# assign_amoutn_transform =  {'rule_type':'static',
#                   'function':'Assign',
#                   'parameters':{'assign_table':{'table':'ocr', 'column':'Amount'}, 
#                                 'assign_value':{'source':'rule', 'value':amount_transform}
#                 }
#              }

where_rule = {'rule_type': 'static',
    'function': 'WhereClause',
    'parameters': {
            'data_frame':{'source':'rule', 'value': amount_transform},
            'from_table': 'ocr',
            'column':'Amount',
            'lookup_filters':[
                {
                    'column_name': 'DR_CR',
                    'lookup_operator' : '==',
                    'compare_with':  {'source':'input', 'value': 'D'}
                }
                
            ]

        }
}


assign_where_transform =  {'rule_type':'static',
                  'function':'Assign',
                  'parameters':{'assign_table':{'table':'ocr', 'column':'Amount'}, 
                                'assign_value':{'source':'rule', 'value':where_rule}
                }
             }

rules = [read_rule, add_amount, add_filetered_df_rule, add_matched_assign_df_rule, add_dr_cr_assign_df_rule, assign_where_transform]
rules= [read_rule]
a = BusinessRules('1234',rules,{},decision = True)
(a.evaluate_business_rules())



C:\Users\Algonox\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:38: DtypeWarning: Columns (42,48,51,65) have mixed types. Specify dtype option on import or set low_memory=False.


{}

In [74]:
def writeToCsv(df, file_path, required_standard_mapping):
    """Write the dataframe to csv"""
    try:

        # first find whether any csv exists with the file_name
        existing_df = pd.read_csv(file_path)

        mode = 'a'
        headers = False
        df.to_csv(file_path, columns=required_columns, mode=mode, header=headers, index=False)
    except:
        # no sample file exists
        headers = True
        mode = 'w'
        
        # rename the columns in the dataframe to standardized columns            
        df = df.rename(columns=required_standard_mapping)
        
        required_columns = required_standard_mapping.values()

        df.to_csv(file_path, columns=required_columns, mode=mode, header=headers, index=False)
                
    return "Written to csv successfully"

In [49]:
sample = pd.read_csv('sample.csv')

C:\Users\Algonox\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (42,48,51,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [75]:
list(sample.columns)
s_req_ = {'Investor type' : 'TYPE'}
file_path = 'C:\\Users\\Algonox\\Desktop\\sample_small.csv'
df = sample.head(5)
writeToCsv(df, file_path, s_req_)
# df.re

'Written to csv successfully'

In [52]:
sample.head(5).to_csv('C:\\Users\\Algonox\\Desktop\\sample_small.csv')

In [55]:
a = [1,2,3]
b = [1,2,3]

c = {e:f for (e,f) in zip(a,b)}
c

{1: 1, 2: 2, 3: 3}

In [59]:
sample.head(5)
sample.head(5).to_csv('C:\\Users\\Algonox\\Desktop\\sample_small_.csv', header=False)

In [40]:
b = a.data_source['ocr'].head(5)

In [44]:
b['Investor type']

0                INDIVIDUAL
1                INDIVIDUAL
2                INDIVIDUAL
3    HINDU UNDIVIDED FAMILY
4    HINDU UNDIVIDED FAMILY
Name: Investor type, dtype: object

In [26]:
a = "12345"

In [37]:
start = -2
end = None
a[start:end]

'45'